## Getting Inshots Summaries

In [ ]:
!pip install selenium
!pip3 install newspaper3k
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 9.6 MB/s 
     |████████████████████████████████| 384 kB 41.3 MB/s 
     |████████████████████████████████| 140 kB 43.8 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 8.4 MB/s 
     |████████████████████████████████| 93 kB 2.8 MB/s 
     |██

In [ ]:
import sys
from bs4 import BeautifulSoup
from lxml import etree
import requests
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import time
import json
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
def getSource(elem):
  return elem.find_all('div', class_='read-more')[0].a.text, elem.find_all('div', class_='read-more')[0].a['href']

def getHeadline(elem):
  return elem.find_all('div', class_='news-card-title')[0].span.text

def getSummary(elem):
  return elem.find_all('div', class_='news-card-content')[0].text

In [ ]:
def addTime_and_Author(elem):
  author = elem.find_all('span', class_="author")[0].text
  timestamp = elem.find_all('span', class_="time")[0]['content']

  return timestamp, author

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.get("https://inshorts.com/en/read")

html = ""
for i in tqdm(range(2000)):
  try:
    element = WebDriverWait(wd, 240).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="load-more-btn"]')))
    element.click()
  except:
    break
  if (i %100 ==0 ) and (i!=0):
    html = wd.page_source

  6%|▌         | 116/2000 [03:32<57:23,  1.83s/it]


In [ ]:
html = wd.page_source
soup = BeautifulSoup(html)
wd.close()

In [ ]:
data = []
exceptions = []
for elem in tqdm(soup.find_all('div', class_='news-card z-depth-1')):
  try:
    source, source_url = getSource(elem)
    timestamp, author = addTime_and_Author(elem)
    if getHeadline(elem)!= "":
      data.append((getHeadline(elem), getSummary(elem), source, source_url, timestamp, author))
  except IndexError:
    exceptions.append(elem)

100%|██████████| 3113/3113 [00:05<00:00, 537.84it/s]


## Getting original news data

In [ ]:
from newspaper import Article
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
def get_article_data(url):
  try:
    article = Article(url)
    article.download()
    article.parse()
    html = article.html
    text = article.text
    return html, text
  except Exception as e:
    return ('Error -' + str(e), "")

In [ ]:
def write_to_df(df):

  df_org = pd.read_csv("/content/drive/MyDrive/InShorts/newsData.csv", sep='|')

  final = pd.concat([df_org, df])
  final = final.drop_duplicates()

  final.to_csv("/content/drive/MyDrive/InShorts/newsData.csv", sep='|', index=False)

  return final

In [ ]:
df = pd.DataFrame(data, columns=['Headline', 'Summary', 'Source', 'URL', 'TimeStamp', 'Author'])
df.head()

In [ ]:
df["html"], df["text"] = zip(*df['URL'].progress_apply(get_article_data))

100%|██████████| 2760/2760 [48:27<00:00,  1.05s/it]


In [ ]:
def fetch_news_properly(html, rule):
  soup = BeautifulSoup(html)
  if "class_" in rule.keys():
    data = soup.find_all(rule['tag'], class_=rule['class_'])
    if data != []:
      if len(data) == 1:
        return data[0].text
      else:
        pass
    else:
      return ''

rule = {'Press Trust of India': {'class_':"fulstorytext", 'tag': 'p'}}

In [ ]:
df['text'] = df.apply(lambda row: fetch_news_properly(row['html'], rule['Press Trust of India']) if row['Source']=='Press Trust of India' else row['text'], axis=1)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/InShorts/newsData.csv', sep='|')

news_df = pd.concat([df1, df[['Headline',
  'Summary',
  'Source',
  'URL',
  'TimeStamp',
  'Author',
  'text']]])

news_df.to_csv('/content/drive/MyDrive/InShorts/newsData.csv', index=False, sep='|')

In [ ]:
news_df.head()

,Headline,Summary,Source,URL,TimeStamp,Author,text
0,Forced to file FIR: MP health official after 3...,"\nA medical officer in Madhya Pradesh's Bhind,...",The Quint,https://www.thequint.com/amp/story/news/india/...,2022-08-24T05:15:01.000Z,Atul Mishra,A government official on whose complaint the p...
1,CBI raids 2 RJD leaders in Bihar in land for r...,\nCBI on Wednesday morning raided two Rashtriy...,Twitter,https://twitter.com/ANI/status/156227508796093...,2022-08-24T03:54:55.000Z,Atul Mishra,JavaScript is not available.\n\nWe’ve detected...
2,"India reports 10,649 new COVID-19 cases in 24 ...","\nIndia has reported 10,649 new COVID-19 cases...",PIB,https://pib.gov.in/PressReleasePage.aspx?utm_c...,2022-08-24T04:38:33.000Z,Apaar Sharma,Ministry of Health and Family Welfare COVID-19...
3,"Sonali said something fishy is going on, she c...",\nLate BJP leader Sonali Phogat's sister Rupes...,ANI,https://twitter.com/ANI/status/156225624415261...,2022-08-24T03:52:58.000Z,Apaar Sharma,JavaScript is not available.\n\nWe’ve detected...
4,Meghan reveals she had to continue royal tour ...,\nMeghan Markle has revealed how she and Princ...,The Independent,https://www.independent.co.uk/life-style/royal...,2022-08-24T04:58:16.000Z,Ridham Gambhir,Stay ahead of the trend in fashion and beyond ...


## Data Cleaning

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
news_df = pd.read_csv("/content/drive/MyDrive/InShorts/newsData.csv", sep='|')
news_df.shape

(23259, 7)

In [ ]:
# Remove all urls containing twitter.com and youtube.com, as they need to be scrapped differently
news_df = news_df[news_df['URL'].str.contains(r'(twitter|facebook|youtube|Cleartrip|instagram|chess|youtu|flipkart|jigsawacademy|inshorts|wimbledon|vired|birmingham2022|amazon|shoppersstop|bseindia|onlinemanipal|usopen)\.(in|com|be|org)')==False]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [ ]:
# finding count of news sources
source_dict = dict(zip(news_df['Source'].value_counts().index.tolist(), news_df['Source'].value_counts().values.tolist()))

# dropping sources which have a count of 1. These are most likely advertisement cards on InShorts
sources_to_drop = [k for k,v in source_dict.items() if v==1]

# adding additional sources which are ads, such as Scaler
sources_to_drop += ['Scaler', 'Goethe Institut', 'MOHFW', 'MoHFW', 'Abbot']

print("Found a total of {} cards which are Advertisements".format(len(sources_to_drop)))
news_df = news_df[~news_df['Source'].isin(sources_to_drop)]

source_dict = dict(zip(news_df['Source'].value_counts().index.tolist(), news_df['Source'].value_counts().values.tolist()))

Found a total of 169 cards which are Advertisements


In [ ]:
source = 'Times Now'
indices = source_dict[source]
np.random.seed(2)
sample = np.random.randint(low=0, high=indices, size=4)
for index in sample:
  print("URL: {}".format(news_df[news_df['Source']==source].iloc[index, 3]))
  print()
  print(index)
  print()
  print(news_df[news_df['Source']==source].iloc[index, 6])
  print()
  print()
  print("*"*100)
  print("*"*100)
  print()
  print()

URL: https://www.timesnownews.com/india/anti-national-slogans-raised-at-jammus-mam-college-after-pakistans-win-over-afghanistan-in-asia-cup-article-94077280/amp?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts 

1192

Anti-national slogans raised at Jammu’s MAM College after Pakistan’s win over Afghanistan in Asia Cup Asia Cup: Pakistan won the match yesterday and entered the finals to take on Sri Lanka over the weekend. Pakistan’s win also meant India are out of contention for a place in the final match. TN National Desk

Anti-national slogans raised at Jammu’s MAM College after Pakistan’s win over Afghanistan in Asia Cup Photo : ANI

Srinagar: Angry scenes were witnessed at the Angry scenes were witnessed at the MAM College in Jammu on Thursday as students staged a protest against anti-national slogans allegedly raised at the institution yesterday. It has been reported that a section of the students raised anti-India slogans at the MAM College following the Asia Cup 2

In [ ]:
news_df['len'] = news_df['text'].apply(lambda x: len(str(x)))
news_df = news_df[news_df['len']>3]

In [ ]:
news_df['clean_summary'] = news_df['Summary'].apply(lambda x: x.split('\n')[1])
news_df['text'] = news_df['text'].apply(lambda x: x.replace('\n', ''))
news_df.head()

,Headline,Summary,Source,URL,TimeStamp,Author,text,len,clean_summary
0,Forced to file FIR: MP health official after 3...,"\nA medical officer in Madhya Pradesh's Bhind,...",The Quint,https://www.thequint.com/amp/story/news/india/...,2022-08-24T05:15:01.000Z,Atul Mishra,A government official on whose complaint the p...,536,"A medical officer in Madhya Pradesh's Bhind, R..."
2,"India reports 10,649 new COVID-19 cases in 24 ...","\nIndia has reported 10,649 new COVID-19 cases...",PIB,https://pib.gov.in/PressReleasePage.aspx?utm_c...,2022-08-24T04:38:33.000Z,Apaar Sharma,Ministry of Health and Family Welfare COVID-19...,742,"India has reported 10,649 new COVID-19 cases a..."
4,Meghan reveals she had to continue royal tour ...,\nMeghan Markle has revealed how she and Princ...,The Independent,https://www.independent.co.uk/life-style/royal...,2022-08-24T04:58:16.000Z,Ridham Gambhir,Stay ahead of the trend in fashion and beyond ...,4990,Meghan Markle has revealed how she and Prince ...
5,I'll kill myself if athletes complain: UP offi...,\nAfter UP Olympic Association's Secretary Gen...,The Bridge,https://thebridge.in/amp/others/lucknow-olympi...,2022-08-24T04:24:12.000Z,Anmol Sharma,A series of photos showing highly-placed sport...,2472,After UP Olympic Association's Secretary Gener...
6,Casemiro receives text from Real Madrid star's...,\nAfter midfielder Casemiro joined Manchester ...,Sportskeeda,https://www.sportskeeda.com/amp/football/news-...,2022-08-24T05:15:36.000Z,Anmol Sharma,Real Madrid midfielder Federico Valverde's gir...,2974,After midfielder Casemiro joined Manchester Un...


In [ ]:
cleaning_data_dict = {"Hindustan Times": ["{{^userSubscribed}} {{/userSubscribed}}",  "SHARE THIS ARTICLE ON"], 
                      "Times Now": ['Representational image.', 'Image:', ' TikTok/@', 'Advertising Advertising', 'Photo:'],
                      "Reuters": ['Our Standards: The Thomson Reuters Trust Principles.', 'Reporting by', 'Register now for FREE unlimited access to Reuters.com Register'],
                      "The Print": ['Also read:', 'This report is auto-generated'],
                      "News18": ['Read the Latest News and Breaking News here', 'Read all the Latest News and Breaking News here', 'Read all the Latest News, Breaking News, watch Top Videos and Live TV here.', 'Advertisement'],
                      "Free Press Journal": ['Advertisement', 'ALSO READ'],
                      "Moneycontrol": ["Read more:"],
                      "ANI News": [],
                      "Sportskeeda": ["Ad", "Quick Links", "Edited by", "last_line"],
                      "The Siasat Daily": [],
                      "Press Trust of India": [], 
                      "The Associated Press": ['ADVERTISEMENT'],
                      "PINKVILLA": [],
                      "LatestLY": ["Watch Video:", "Representative Image", "View Tweet Here:", "Representational Picture"],
                      "ANI": [],
                      "Financial Express": ["Also Read|", "Also Read:", "Watch Video |", "Watch Video|", "Watch Video:"],
                      "The Quint": [],
                      "CricTracker": []}

In [ ]:
# save a copy of cleaned data
news_df.to_csv('/content/drive/MyDrive/InShorts/newsDataCleaned.csv', sep='|', index=False)

## Model Building

In [ ]:
! pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.2 MB/s 
     |████████████████████████████████| 5.5 MB 68.3 MB/s 
     |████████████████████████████████| 212 kB 60.5 MB/s 
     |████████████████████████████████| 163 kB 75.8 MB/s 
     |████████████████████████████████| 95 kB 6.4 MB/s 
     |████████████████████████████████| 115 kB 77.7 MB/s 
     |████████████████████████████████| 127 kB 74.5 MB/s 
     |████████████████████████████████| 7.6 MB 47.3 MB/s 
     |████████████████████████████████| 115 kB 59.5 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=7d3fbd6918715259a031bfcd0afb5577863e069a1a2b74540c5a644c75300994
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    

In [ ]:
# imports
from datasets import Dataset, DatasetDict, load_metric
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig
import pandas as pd
import nltk
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# load the dataset
news_df = pd.read_csv('/content/drive/MyDrive/InShorts/newsDataCleaned.csv', sep='|')
news_df = news_df[['text', 'clean_summary']]

# found some None values for some reason, dropping them here temporarily
news_df = news_df[~news_df['text'].isna()]
news_df.dropna(inplace=True)
train, test = train_test_split(news_df, test_size=0.1, random_state=12)

train_data = Dataset.from_pandas(train)
test_data = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['train'] = train_data
dataset['test'] = test_data

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'clean_summary', '__index_level_0__'],
        num_rows: 17832
    })
    test: Dataset({
        features: ['text', 'clean_summary', '__index_level_0__'],
        num_rows: 1982
    })
})

In [ ]:
# Tokenizing the data

model_to_use = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_to_use)

if model_to_use.startswith('t5'):
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 2048
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["clean_summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/18 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# function to compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Using the model and fine tuning

model = AutoModelForSeq2SeqLM.from_pretrained(model_to_use)
metric = load_metric("rouge")

batch_size = 4
model_name = model_to_use.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-inshots",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/d78aea13fa7ecd06c29e3e46195d6341255065d5/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_b

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: clean_summary, __index_level_0__, text. If clean_summary, __index_level_0__, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17832
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 22290
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.754800,1.579784,24.305100,13.574200,20.949200,22.417100,19.000000
2,1.694800,1.535118,24.756000,13.874100,21.285200,22.817300,19.000000
3,1.677000,1.513046,24.768500,13.915900,21.288700,22.858800,19.000000
4,1.666500,1.505894,24.995700,14.048600,21.488300,23.023700,19.000000
5,1.641000,1.504159,24.918000,13.987700,21.373600,22.930300,19.000000


Saving model checkpoint to t5-small-finetuned-inshots/checkpoint-500
Configuration saved in t5-small-finetuned-inshots/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-inshots/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-inshots/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-inshots/checkpoint-500/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-inshots/checkpoint-1000
Configuration saved in t5-small-finetuned-inshots/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-inshots/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-inshots/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-inshots/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-inshots/checkpoint-1500
Configuration saved in t5-small-finetuned-inshots/checkpoint-1500/config.json
Model wei

TrainOutput(global_step=22290, training_loss=1.7047613897789964, metrics={'train_runtime': 7119.9615, 'train_samples_per_second': 12.523, 'train_steps_per_second': 3.131, 'total_flos': 2.0903537007919104e+16, 'train_loss': 1.7047613897789964, 'epoch': 5.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/InShorts/model')

Saving model checkpoint to /content/drive/MyDrive/InShorts/model
Configuration saved in /content/drive/MyDrive/InShorts/model/config.json
Model weights saved in /content/drive/MyDrive/InShorts/model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/InShorts/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/InShorts/model/special_tokens_map.json


In [ ]:
from transformers import pipeline

hub_model_id = "/content/drive/MyDrive/InShorts/model"
summarizer = pipeline("summarization", model=hub_model_id)

In [ ]:
summarizer('Washington [US], October 31 (ANI): US President Joe Biden lost his temper with Volodymyr Zelenskyy in June during a phone conversation when he asked for more military aid, NBC News reported on Monday, citing sources familiar with the call. The report said Biden routinely calls Zelenskyy when the US announces new aid packages for Ukraine. But the June call was different. Biden had barely finished informing Zelenskyy that he had approved another USD 1 billion in military assistance for Ukraine when his counterpart started asking for extra help Kyiv needs but isn’t getting, the report said. Biden raised his voice, and as per the NBC report said Zelenskyy could “show a little more gratitude.” Prior to the June 15 phone call, Biden’s dissatisfaction with Zelenskyy had been building for weeks, the sources said. According to them, the US president and a number of his aides believed that Washington was doing everything possible and as quickly as possible, but Zelenskyy continued to publicly pay attention only to what was not being done. After Zelenskyy was rebuffed during the June call, Zelenskyy publicly delivered a video message thanking Biden for the assistance and defusing the tensions. “I had an important conversation with US President Biden today,” NBC quoted Ukraine’s president in videotaped remarks. “I am grateful for this support. It is especially important for our defence in Donbas.” The United States has been a leading provider of security assistance to Ukraine, particularly since the start of the Russia-Ukraine conflict on February 24. This report on the Biden-Zelenskyy phone call comes two days after Washington announced USD 275 million in additional military assistance for Ukraine. “This drawdown will bring the total US military assistance for Ukraine to an unprecedented level of more than USD 18.5 billion since the beginning of the Administration,” the US State Department said in a statement. The United States, in 2022, provided more advanced defence equipment to Ukraine, as well as greater amounts of previously provided equipment, according to a Congressional Research Service report. According to Pentagon, US security assistance committed to Ukraine, includes High Mobility Artillery Rocket Systems, Stinger anti-aircraft systems, Javelin anti-armour systems and Mi-17 helicopters. Ukrainian officials have sought to acquire other advanced systems, including fighter aircraft, anti-ship, and additional air defence and anti-missile capabilities. (ANI) This report is auto-generated from ANI news service. ThePrint holds no responsibility for its content.')

Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "US President Joe Biden lost his temper with US President Volodymyr Zelenskyy during a phone conversation in June when he asked for more military aid, NBC News reported on Monday. Biden had barely finished informing him that he had approved another USD 1 billion in military assistance for Ukraine when his counterpart started asking for extra help Kyiv needs but isn't getting, the report said."}]